In [4]:
import os
import torch
import numpy as np
from plyfile import PlyData, PlyElement

from mast3r.model import AsymmetricMASt3R

import mast3r.utils.path_to_dust3r
from dust3r.inference import inference
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from dust3r.demo import get_3D_model_from_scene
from dust3r.utils.device import to_numpy

In [ ]:
# 데이터 설정
target_data = "scan114"
# source_path = f"/home/airlabs/Dataset/LLFF/llff_8/{target_data}"
source_path = f'/home/airlabs/Dataset/DTU/dtu_4/{target_data}'

In [ ]:
device = 'cuda'
schedule = 'cosine'
lr = 0.01
niter = 300

model_name = "naver/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric"
model = AsymmetricMASt3R.from_pretrained(model_name).to(device)

# Set Path
input3_path = os.path.join(source_path, '3')
output_path = os.path.join(source_path, "dust3r")

In [ ]:
images = load_images(source_path, size=512)
pairs = make_pairs(images, scene_graph='oneref-0', prefilter=None, symmetrize=True)
output = inference(pairs, model, device, batch_size=1)

## PLY 최적화

In [1]:
import open3d as o3d
ply_path = "/home/airlabs/Dataset/DTU/dtu_4/scan82/dust3r_test/ply/output.ply"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd = o3d.io.read_point_cloud(ply_path)

In [3]:
pcd = o3d.io.read_point_cloud(ply_path)
cl, ind = pcd.remove_radius_outlier(nb_points=16, radius=0.05)
pcd_radius = pcd.select_by_index(ind)    
cl, ind = pcd_radius.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd_statistical = pcd_radius.select_by_index(ind)

In [ ]:
o3d.visualization.draw_geometries([pcd_radius])